In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from keras import models
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from keras import regularizers

In [2]:
x_test = pd.read_excel('C:\\Users\\Daniel\\Documents\\Flatiron\\Capstone\\sports_predictions\\stats.xlsx')

In [3]:
x_test

,Home Points For,Home Points Against,Home eFG%,Home FTr,Home ORB%,Home DRB%,Home AST%,Home STL%,Home BLK%,Home TOV%,...,Away DRB%,Away AST%,Away STL%,Away BLK%,Away TOV%,Away ORTG,Away DRtg,Away GP,Away Win Rate,Vegas Prediction
0,114.5,117.5,0.519,0.230,23.2,75.0,61.9,9.5,4.9,11.7,...,76.5,55.5,7.5,6.3,11.9,112.2,111.0,12,0.833333,-1.5
1,108.4,108.2,0.502,0.226,21.0,77.2,56.9,8.5,6.5,12.2,...,78.2,54.9,7.6,4.5,12.5,107.9,107.7,12,0.714286,-1.0
2,105.0,104.3,0.525,0.243,24.4,73.3,63.3,7.6,3.7,14.8,...,79.9,58.9,8.6,5.1,13.4,98.0,108.4,10,0.200000,4.5
3,103.8,111.3,0.512,0.294,21.6,76.6,56.4,8.3,4.4,16.9,...,81.4,57.5,8.0,4.8,12.5,110.7,109.8,12,0.000000,-6.5
4,109.7,114.2,0.508,0.180,27.0,77.9,60.8,10.5,5.9,12.4,...,78.0,63.9,8.9,4.8,12.5,109.4,107.6,10,0.670000,-2.0
5,99.1,106.7,0.473,0.241,21.4,75.7,56.4,7.1,4.1,13.8,...,76.2,56.1,6.9,5.1,12.6,109.8,110.3,12,0.500000,-4.5
6,100.5,111.9,0.474,0.201,25.6,78.4,62.0,7.7,4.4,13.9,...,78.5,62.3,6.3,5.5,14.3,106.7,103.5,12,0.670000,-9.0
7,110.0,107.9,0.523,0.185,20.6,77.3,64.6,9.4,5.8,11.4,...,79.6,54.9,6.9,4.5,12.1,105.3,108.4,11,0.400000,-2.5
8,101.8,98.5,0.523,0.180,20.5,80.2,59.1,8.5,4.4,14.1,...,113.2,56.2,6.8,3.7,11.8,109.1,113.2,12,0.125000,4.0
9,115.9,102.3,0.560,0.233,22.4,81.1,70.7,10.5,4.8,13.9,...,78.5,56.9,8.8,5.1,11.3,111.7,104.3,11,0.750000,5.5


In [4]:
df = pd.read_csv('avg_by_game')
odds = pd.read_csv('nba_game_odds')

In [5]:
data = df.set_index(df['Unnamed: 0'], drop=True)
data.index.rename('GameID', inplace=True)
data.drop(columns='Unnamed: 0', inplace=True)

odds.set_index(odds['GameID'], drop=True, inplace=True)

In [6]:
data.sort_index(inplace=True)
odds.sort_index(inplace=True)
data['Home Wins By (Vegas)'] = odds['Home Win By (Vegas)']

In [7]:
for i in range(len(data)):
    data['Home Wins By (Vegas)'][i] = odds['Home Win By (Vegas)'][i]

<ipython-input-7-3759b8bb49ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Home Wins By (Vegas)'][i] = odds['Home Win By (Vegas)'][i]


In [8]:
results = pd.DataFrame(columns=['linreg', 'forest', 'gb', 'NN', 'Vegas Prediction'],
                      index = range(len(x_test)))

In [9]:
x = data.drop(columns='Home Win By')
y = data['Home Win By']

In [10]:
linreg = LinearRegression()
forest = RandomForestRegressor(max_depth=10, min_samples_leaf=3, min_samples_split=3, n_estimators=200)
gb = GradientBoostingRegressor(learning_rate=0.2, max_depth=1, min_samples_leaf=5, min_samples_split=3, n_estimators=200, random_state=42)
ml_models = [linreg, forest, gb]

In [11]:
def run_model(x, y, x_test, model):

    model.fit(x, y)
    y_test_pred = model.predict(x_test)
    
    return y_test_pred

In [12]:
nn = models.Sequential()
nn.add(Dense(500, input_dim=29, activation= 'relu'))
nn.add(Dense(250, activation= 'relu'))
nn.add(Dense(125, activation= 'relu'))

nn.add(Dense(1, activation='linear'))

nn.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])

In [13]:
nn.fit(x, y, epochs=200, verbose=2, batch_size=300)
test_preds = nn.predict(x_test)
results['NN'] = test_preds

Epoch 1/200
39/39 - 1s - loss: 239.9442 - mean_squared_error: 239.9442
Epoch 2/200
39/39 - 0s - loss: 142.9162 - mean_squared_error: 142.9162
Epoch 3/200
39/39 - 0s - loss: 132.3006 - mean_squared_error: 132.3006
Epoch 4/200
39/39 - 0s - loss: 133.6733 - mean_squared_error: 133.6733
Epoch 5/200
39/39 - 0s - loss: 135.3154 - mean_squared_error: 135.3154
Epoch 6/200
39/39 - 0s - loss: 130.4439 - mean_squared_error: 130.4439
Epoch 7/200
39/39 - 0s - loss: 129.8885 - mean_squared_error: 129.8885
Epoch 8/200
39/39 - 0s - loss: 129.2193 - mean_squared_error: 129.2193
Epoch 9/200
39/39 - 0s - loss: 129.8159 - mean_squared_error: 129.8159
Epoch 10/200
39/39 - 0s - loss: 131.7289 - mean_squared_error: 131.7289
Epoch 11/200
39/39 - 0s - loss: 130.1725 - mean_squared_error: 130.1725
Epoch 12/200
39/39 - 0s - loss: 128.1828 - mean_squared_error: 128.1828
Epoch 13/200
39/39 - 0s - loss: 127.7758 - mean_squared_error: 127.7758
Epoch 14/200
39/39 - 0s - loss: 128.2872 - mean_squared_error: 128.2872
E

Epoch 115/200
39/39 - 0s - loss: 124.8036 - mean_squared_error: 124.8036
Epoch 116/200
39/39 - 0s - loss: 124.5866 - mean_squared_error: 124.5866
Epoch 117/200
39/39 - 0s - loss: 126.1606 - mean_squared_error: 126.1606
Epoch 118/200
39/39 - 0s - loss: 125.6372 - mean_squared_error: 125.6372
Epoch 119/200
39/39 - 0s - loss: 124.5519 - mean_squared_error: 124.5519
Epoch 120/200
39/39 - 0s - loss: 124.7294 - mean_squared_error: 124.7294
Epoch 121/200
39/39 - 0s - loss: 127.0800 - mean_squared_error: 127.0800
Epoch 122/200
39/39 - 0s - loss: 127.7427 - mean_squared_error: 127.7427
Epoch 123/200
39/39 - 0s - loss: 125.7402 - mean_squared_error: 125.7402
Epoch 124/200
39/39 - 0s - loss: 125.2069 - mean_squared_error: 125.2069
Epoch 125/200
39/39 - 0s - loss: 126.2737 - mean_squared_error: 126.2737
Epoch 126/200
39/39 - 0s - loss: 126.2484 - mean_squared_error: 126.2484
Epoch 127/200
39/39 - 0s - loss: 124.6292 - mean_squared_error: 124.6292
Epoch 128/200
39/39 - 0s - loss: 124.6143 - mean_sq

In [14]:
column_names = ['linreg', 'forest', 'gb', 'NN']

In [15]:
for i, model in enumerate(ml_models):
    preds = run_model(x, y, x_test, model)
    results[column_names[i]] = preds

In [16]:
results['Vegas Prediction'] = x_test['Vegas Prediction']

In [17]:
results

,linreg,forest,gb,NN,Vegas Prediction
0,-3.937607,-7.707503,-5.952967,-3.557056,-1.5
1,-1.119679,-7.025503,-9.127164,-0.848410,-1.0
2,6.508230,5.304852,5.571153,5.238664,4.5
3,-8.920161,1.243100,-5.785026,-7.645515,-6.5
4,-5.510217,-3.412284,-4.320616,-6.085435,-2.0
5,-7.221938,-4.460187,-4.915437,-7.592675,-4.5
6,-12.624729,-11.423366,-13.072220,-12.680906,-9.0
7,1.987836,-0.367212,-0.007495,3.159029,-2.5
8,-1.512996,7.227273,7.968087,-1.174496,4.0
9,5.059953,6.875696,4.456066,3.896121,5.5
